Read file line by line
Run query on each line
Write result out to file

In [6]:
import re

RESULT_FILE = 'query_results.txt'

In [7]:
def file_reader(file):
    with open(file) as infile:
        for line in infile:
            print(line)

In [8]:
def write_result_to_file(result):
    # write to file here.
    file = open(RESULT_FILE, 'a')
    file.write(result +'\n')
    file.close()

In [9]:
def query(string, *args):
    """
    Takes a string from the file as first parameter.
    The args parameter is the query it self in the format of:
    ['cat', [2, 4], 'hat']
    """
    if args is None:
        return
    
    for query in args:
        # seperate query into parameters
        start_word = query[0]
        end_word = query[2]
        lower_bound = query[1][0]
        upper_bound = query[1][1]
        
        # build regexs
        start_word_regex = r"\b" + re.escape(start_word) + r"\b"
        end_word_regex = r"\b" + re.escape(end_word) + r"\b"
        
        # find substring positions
        start_word_pos = re.search(start_word_regex, string)
        end_word_pos = re.search(end_word_regex, string)

        # calc distance between words
        char_between = end_word_pos.start() - start_word_pos.end()

        if char_between >= lower_bound and char_between <= upper_bound:
            result = string[start_word_pos.start(): end_word_pos.end()]
            # write result to file
            write_result_to_file(result)

In [10]:
test_string = 'I have a really nice cat in hat at home'
test_query = ['cat', [2, 4], 'hat']
test_query2 = ['I', [2, 6], 'a']
query(test_string, test_query, test_query2)